In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.ensemble import BaggingClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier, RidgeClassifierCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.neural_network import MLPClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.ensemble import VotingClassifier
from sklearn.utils.class_weight import compute_class_weight
from catboost import  CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from IPython.display import clear_output
from tqdm import tqdm
import numpy as np
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE, SMOTENC, SMOTEN
from imblearn.under_sampling import ClusterCentroids, EditedNearestNeighbours, RepeatedEditedNearestNeighbours, AllKNN, NearMiss, InstanceHardnessThreshold, NeighbourhoodCleaningRule, OneSidedSelection, TomekLinks
from imblearn.combine import SMOTETomek, SMOTEENN

In [32]:
from sklearn.utils.class_weight import compute_class_weight
from catboost import  CatBoostClassifier
from sklearn.neural_network import MLPClassifier

train = pd.read_csv("./data/train.csv", encoding="utf-8-sig")
test = pd.read_csv("./data/test.csv", encoding="utf-8-sig")
sample = pd.read_csv("./data/sample_submission.csv", encoding="utf-8-sig")

from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder

# 수치형 및 범주형 변수 구분
numerical_cols = [col for col in train.columns if train[col].dtype in ['int64', 'float64'] and col not in ['target']]
numerical_cols.remove('payment_pattern')
categorical_cols = [col for col in train.columns if train[col].dtype == 'object' and col not in ['user_id']]
categorical_cols.append('payment_pattern')
# 데이터 스케일링을 위한 StandardScaler 인스턴스 생성
scaler = StandardScaler()

# 수치형 데이터에 대해 스케일링 적용
train[numerical_cols] = scaler.fit_transform(train[numerical_cols])
test[numerical_cols] = scaler.transform(test[numerical_cols])

# 레이블 인코딩을 위한 LabelEncoder 인스턴스 생성 및 적용
label_encoders = {}
for col in categorical_cols:
    label_encoders[col] = LabelEncoder()
    train[col] = label_encoders[col].fit_transform(train[col])
    test[col] = label_encoders[col].transform(test[col])

# 전처리 후 학습 데이터 확인
train.head()

# 특성과 타겟 변수 분리
X = train.drop(columns=['user_id', 'target'])  # user_id는 제외하고 특성 데이터로 사용
y = train['target']

# 학습 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
#X_train, y_train = OneSidedSelection().fit_resample(X_train, y_train)

In [45]:
from sklearn.metrics import accuracy_score
# 훈련 데이터를 기반으로 클래스 가중치 계산 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, y_train = OneSidedSelection().fit_resample(X_train, y_train)
class_weight  = compute_class_weight( 'balanced' , classes=[ 0 , 1 ], y=y_train)
weights = {i: w for i, w in enumerate(class_weight)}

cat = CatBoostClassifier(depth=6, iterations=700, learning_rate=0.1, random_state=42, class_weights=weights, verbose=0)
mp = MLPClassifier(activation='tanh', alpha=0.01, hidden_layer_sizes=100, learning_rate_init=0.01, solver='adam', random_state=42)
xgb = XGBClassifier(learning_rate=1.0, max_depth=7, n_estimators=50, random_state=42, subsample=0.5)

cat.fit(X_train, y_train)
mp.fit(X_train, y_train)
xgb.fit(X_train, y_train)

cat_predictions = cat.predict(X_train)
mp_predictions = mp.predict(X_train)
xgb_predictions = xgb.predict(X_train)

from sklearn.linear_model import LogisticRegression

pred = np.array([cat_predictions, mp_predictions, xgb_predictions])
pred = np.transpose(pred)

lr_final = LogisticRegression()
lr_final.fit(pred, y_train)  # 학습 데이터를 사용하여 최종 모델 학습

cat_test_predictions = cat.predict(X_val)
mp_test_predictions = mp.predict(X_val)
xgb_test_predictions = xgb.predict(X_val)

# 검증 세트로 최종 모델 예측
pred_test = np.array([cat_test_predictions, mp_test_predictions, xgb_test_predictions])
pred_test = np.transpose(pred_test)
final_test = lr_final.predict(pred_test)
val_f1 = f1_score(y_val, final_test, average='macro', zero_division=0)
print(f'Validation F1-score: {val_f1}')
print(classification_report(y_val, final_test, zero_division=0))

Validation F1-score: 0.50341624614835
              precision    recall  f1-score   support

           0       0.39      0.49      0.43       758
           1       0.63      0.53      0.57      1242

    accuracy                           0.51      2000
   macro avg       0.51      0.51      0.50      2000
weighted avg       0.54      0.51      0.52      2000



In [25]:
cat_test = cat.predict(test)
mp_test = mp.predict(test)
xgb_test = xgb.predict(test)

test_pred = np.array([cat_test, mp_test, xgb_test])
test_pred = np.transpose(test_pred)

test_final = lr_final.predict(test_pred)
print(np.unique(test_final))

[1]
